In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
import seaborn as sns


# Load latest Momoko file
latest_csv = pd.read_csv("combined_parliaments_new_JUN12-2021.csv", index_col=0)

# get the "Name", "Gender", "Office", Birth_year" columns only
data = latest_csv[["Name","Gender","Office","Birth_year"]]

#rename the columns
data.columns = ["Name","Gender","Parliament","YOB"]

#DROP rows that have YOB as null value
data.dropna(subset=["YOB"], inplace=True)
### DROP rows with not a number for YOB
data.drop(data.index[data['YOB'] == "Baby bommer"], inplace=True)
data.drop(data.index[data['YOB'] == "Baby boomer"], inplace=True)
data.drop(data.index[data['YOB'] == "Generation X"], inplace=True)
data.drop(data.index[data['YOB'] == "Millenial"], inplace=True)
# assign int type to YOB 
data["YOB"] = data["YOB"].astype(int)


#reset the index
data.reset_index(drop=True, inplace=True)

#create a column for Parliament_year 
conditions = [
    (data['Parliament'] == 28),
    (data['Parliament'] == 29),
    (data['Parliament'] == 30),
    (data['Parliament'] == 31),
    (data['Parliament'] == 32),
    (data['Parliament'] == 33),
    (data['Parliament'] == 34),
    (data['Parliament'] == 35),
    (data['Parliament'] == 36),
    (data['Parliament'] == 37),
    (data['Parliament'] == 38),
    (data['Parliament'] == 39),
    (data['Parliament'] == 40),
    (data['Parliament'] == 41),
    (data['Parliament'] == 42),
    (data['Parliament'] == 43)
    ]
# create a list of the values we want to assign for each condition
values = [1968,1972,1974,1979,1980,1984,1988,1993,1997,2000,2004,2006,2008,2011,2015,2019]
# create a new column and use np.select to assign values to it using our lists as arguments
data['Parliament_Year'] = np.select(conditions, values)

# create Age@Parliament column
data['Age@Parliament']=data["Parliament_Year"]-data["YOB"]

### set Age_Buckets columns
data["Age_buckets"]=""
for i, k in enumerate(data['Age@Parliament']):   
    if 17<k<31:
        data["Age_buckets"].loc[i]="18-30"
    elif 30<k<41:
        data["Age_buckets"].loc[i]="31-40"
    elif 40<k<51:
        data["Age_buckets"].loc[i]="41-50"
    elif 50<k<61:
        data["Age_buckets"].loc[i]="51-60"
    elif 60<k<71:
        data["Age_buckets"].loc[i]="61-70"
    else:
        data["Age_buckets"].loc[i]="71+"    

data.to_csv('MPs_by_Age.csv')
#Parliaments_by_AgeBuckets = data.drop(['Parliament','YOB','Age@Parliament'], axis=1)
Parliaments_by_AgeBuckets = data.drop(['Parliament','YOB'], axis=1)
#Parliaments_by_AgeBuckets = Parliaments_by_AgeBuckets.groupby(["Parliament_Year","Age_buckets"])["Name"].count().reset_index()
Parliaments_by_AgeBuckets = Parliaments_by_AgeBuckets.groupby(["Parliament_Year","Age_buckets"]).agg({"Age@Parliament":["mean"],"Name":["count"]}).reset_index()
#Parliaments_by_AgeBuckets.rename(columns = {'Name':'MP_count'}, inplace = True)
#Parliaments_by_AgeBuckets.rename(columns = {'Parliament_Year':'Parliament_year'}, inplace = True)
Parliaments_by_AgeBuckets.columns = ["Parliament_year","Age_buckets","Avg_MP_age", "MP_count"]
Parliaments_by_AgeBuckets["Avg_MP_age"] = Parliaments_by_AgeBuckets["Avg_MP_age"].round().astype(int)
Parliaments_by_AgeBuckets = Parliaments_by_AgeBuckets[(Parliaments_by_AgeBuckets["Parliament_year"]>1968)]
Parliaments_by_AgeBuckets.reset_index(drop=True, inplace=True)

Parliaments_by_AgeBuckets.to_csv('Parliaments_by_AgeBuckets.csv')

#Parliaments_by_AgeBuckets = Parliaments_by_AgeBuckets[(Parliaments_by_AgeBuckets["Parliament_year"]>1968)]



census_data = pd.read_csv("Age_Canada_FINAL.csv", index_col=0)
# drop columns we don't need
census_data.drop(['0 – 4', '5 – 9', '10 – 14'], axis = 1, inplace = True)
# convert columns with NAN from Float to Int46 (nullable integer type) - 
# regular int type doesnt support Nan in the values
census_data["90 – 94"] = census_data["90 – 94"].astype('Int64')
census_data["95 – 99"] = census_data["95 – 99"].astype('Int64') 
census_data["> 100"] = census_data["> 100"].astype('Int64') 

census_data["18-30"] = (census_data["15 – 19"]/5)*2+census_data["20 – 24"]+census_data["25 – 29"]+(census_data["30 – 34"]/5)
census_data["18-30"] = census_data["18-30"].apply(int)
census_data["31-40"] = (census_data["30 – 34"]/5)*4+census_data["35 – 39"]+(census_data["40 – 44"]/5)
census_data["31-40"] = census_data["31-40"].apply(int)
census_data["41-50"] = (census_data["40 – 44"]/5)*4+census_data["45 – 49"]+(census_data["50 – 54"]/5)
census_data["41-50"] = census_data["41-50"].apply(int)
census_data["51-60"] = (census_data["50 – 54"]/5)*4+census_data["55 – 59"]+(census_data["60 – 64"]/5)
census_data["51-60"] = census_data["51-60"].apply(int)
census_data["61-70"] = (census_data["60 – 64"]/5)*4+census_data["65 – 69"]+(census_data["70 – 74"]/5)
census_data["61-70"] = census_data["61-70"].apply(int)
census_data = census_data.fillna(0)
census_data["71+"] = (census_data["70 – 74"]/5)*4 + census_data["75 – 79"] + census_data["80 – 84"] + census_data["85 – 89"] + census_data["90 – 94"] + \
                        census_data["95 – 99"] + census_data["> 100"]
census_data["71+"] = census_data["71+"].apply(int) 

census_data =census_data[["18-30", "31-40", "41-50", "51-60", "61-70", "71+"]]
# get census data from 1972 parliament
census_final = census_data.loc[[1972,1974,1979,1980,1984,1988,1993,1997,2000,2004,2006,2008,2011,2015,2019]]
# reswet the index
census_final.reset_index(drop=False, inplace=True)
# rename the column
census_final.rename(columns = {'index':'Parliament_year'}, inplace = True)

Census_by_AgeBuckets = pd.melt(census_final, id_vars="Parliament_year", var_name="Age_buckets", value_name="Voters_count")
Census_by_AgeBuckets.to_csv('Census_data_by_AgeBuckets.csv')


# MERGE BOTH datasets
all_data = pd.merge(Parliaments_by_AgeBuckets, Census_by_AgeBuckets, on=["Parliament_year", "Age_buckets"])

Voters_count_per_year = all_data.groupby("Parliament_year")["Voters_count"].sum().reset_index()
MP_count_per_year = all_data.groupby("Parliament_year")["MP_count"].sum().reset_index()

Total_MP_and_Voters_count_per_year = pd.merge(Voters_count_per_year, MP_count_per_year, on="Parliament_year")

final_data = pd.merge(all_data, Total_MP_and_Voters_count_per_year, on="Parliament_year")

final_data.rename(columns = {'MP_count_x':'MP_count','Voters_count_x':'Voters_count','Voters_count_y':'Total_Voters_count','MP_count_y':'Total_MP_count' }, inplace = True)
final_data["Voters_count_%"] = ((final_data["Voters_count"]/final_data["Total_Voters_count"])*100).round(decimals=1)
final_data["MP_count_%"] = ((final_data["MP_count"]/final_data["Total_MP_count"])*100).round(decimals=1)
final_data.to_csv("Voters_and_MPs_Age_buckets_per_Parliament_June14-2021.csv")

C:\Users\Design\anaconda3\lib\site-packages\ipykernel_launcher.py:18: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Design\anaconda3\lib\site-packages\ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\Design\anaconda3\lib\site-packages\ipykernel_launcher.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi